<a href="https://colab.research.google.com/github/AiJared/DCGAN/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from IPython import display

In [2]:
# set random seed for reproducibility
tf.random.set_seed(42)

# Hyperparameters
BUFFER_SIZE = 60000
BATCH_SIZE = 128
EPOCHS = 200
noice_dim = 100
IMAGE_SIZE = 64
CHANNELS = 3